In [1]:
# TGB is 180 hectares
import numpy as np
import matplotlib.pyplot as plt
import importlib as imp
import Radar_drawn
import os
import matplotlib.patches as patches

In [ ]:
from PIL import Image
# in following line, replace 'gpr_image' with the composite gpr image
background = np.asarray(Image.open('gpr_image.png'))
print(f"background shape: {np.asarray(background).shape}")
image_mask = np.loadtxt(f'union_mask.txt') # not square
mask_x, mask_y = image_mask.shape
image_mask = image_mask[mask_x - mask_y:]
print(f"mask shape: {image_mask.shape}")
# culled_background = background * image_mask

In [ ]:
"""Visualize GPR image, cropped with the region used for comparison"""
f, ax = plt.subplots(figsize=(16, 8), ncols=2)
culled_background = background[:, :, :3] * image_mask[:, :, np.newaxis]
culled_background_flip = 255 - culled_background
ax[0].imshow((culled_background).astype(np.uint8))
ax[1].imshow((culled_background_flip).astype(np.uint8))

In [4]:
project_name = 'TBK_mesh_n=4_depth=10' # name of project, must be same as that used in previous files!
crestlines_lines_mask = np.loadtxt(f'{project_name}_crestlines_lines_mask.txt')
image_lines_composite_mask = np.loadtxt("image_ridges_composite.txt")
mask = np.loadtxt(f'{project_name}_union_mask.txt') # NOTE: same as image_mask
# apply the mask, so we only compare what's inside the area it represents
crestline_mask = crestlines_lines_mask * mask
radar_mask = image_lines_composite_mask * mask

In [5]:
beginning = np.loadtxt(fname=f'{project_name}_crestline_2d_vertices_start.txt')
ending = np.loadtxt(fname=f'{project_name}_crestline_2d_vertices_end.txt')

In [ ]:
pc_max_x = max(beginning[:, 0].max(), ending[:, 0].max())
pc_max_y = max(beginning[:, 1].max(), ending[:, 1].max())
pc_min_x = min(beginning[:, 0].min(), ending[:, 0].min())
pc_min_y = min(beginning[:, 1].min(), ending[:, 1].min())
print(f"bounding box in Point Cloud units: "
      f"\n\tx in {pc_min_x, pc_max_x},"
      f"\n\ty in {pc_min_y, pc_max_y}")

In [ ]:
image_x, image_y = crestline_mask.shape
x_lower_bound = 0
for x in range(image_x):
    if np.sum(crestline_mask[x]) == 0:
        x_lower_bound = x
    else:
        break
x_upper_bound = image_x
for x in range(image_x).__reversed__():
    if np.sum(crestline_mask[x]) == 0:
        x_upper_bound = x
    else:
        break
y_lower_bound = 0
for y in range(image_y):
    if np.sum(crestline_mask[:,y]) == 0:
        y_lower_bound = y
    else:
        break
y_upper_bound = image_y
for y in range(image_y).__reversed__():
    if np.sum(crestline_mask[:,y]) == 0:
        y_upper_bound = y
    else:
        break
print(f"bounding box in pixels: x in {x_lower_bound, x_upper_bound}")
print(f"bounding box in pixels: y in {y_lower_bound, y_upper_bound}")
thicken = 40
x_hi = x_upper_bound + thicken
x_lo = x_lower_bound - thicken
y_hi = y_upper_bound + thicken
y_lo = y_lower_bound - thicken
print(f"displaying cropper box in pixels: x in {x_lo, x_hi}")
print(f"displaying cropper box in pixels: y in {y_lo, y_hi}")

In [ ]:
def irl_to_pixel(irl_length : float):
    return int(irl_length 
        * (x_upper_bound - x_lower_bound) / (204.597716)
        )
test_irl_radius = 0.8
print(f"{test_irl_radius} is {irl_to_pixel(test_irl_radius)} pixels")
h = "#32a852"
def toRGB(h : str):
    return np.asarray(list(int(h.lstrip("#")[i:i+2], 16) for i in (0, 2, 4)))
toRGB(h)

In [ ]:
"""view image mask"""
# f, ax = plt.subplots(figsize=(8, 8))
# print(image_mask[x_lo:x_hi, y_lo:y_hi].shape)
# ax.imshow(image_mask[x_lo:x_hi, y_lo:y_hi])
# print(np.sum(image_mask))
# print(np.size(image_mask) - np.sum(image_mask))
# print(np.size(image_mask))
# rect = patches.Rectangle((thicken, thicken), 
#     (y_upper_bound - y_lower_bound),
#     (x_upper_bound - x_lower_bound),  
#     linewidth=2, edgecolor='r', facecolor='none')
# ax.add_patch(rect)

In [ ]:
"""Good Version"""
imp.reload(Radar_drawn)
# 1.2=11, 1.1=10, 1.0=9, 0.9=8, 0.8=7
irl_radius = 1.2 # current: 1.1
pixel_radius = irl_to_pixel(irl_radius)
# make folder
folder = ""# f"irl={irl_radius}_px={pixel_radius}"
# os.mkdir(folder)
print(f"{irl_radius} in real life is {pixel_radius} in the picture")
comparer = Radar_drawn.Comparer(name=project_name, 
    mask_a=radar_mask[x_lo:x_hi, y_lo:y_hi], 
    mask_b=crestline_mask[x_lo:x_hi, y_lo:y_hi],
    names={},
    background=culled_background_flip[x_lo:x_hi, y_lo:y_hi],
    result_filename=f'TBK_d=11_n=4_irl={irl_radius}_r={pixel_radius}px',
    dir=folder,
    filetype='tiff',
    DPI=600
)
# _ = comparer.CM(
#     true_pos_color=np.asarray([0, 51, 102]), #toRGB("#2ab7ca"),
#     false_pos_color=np.asarray([255, 153, 0]), #toRGB("#fed766"),
#     false_neg_color=np.asarray([128, 128, 128]), #toRGB("#fe4a49"),
#     true_neg_color=np.asarray([50, 50, 50]),
#     background_color=toRGB("#ffffff"),
#     comparison_space=(image_mask).astype(np.uint8)[x_lo:x_hi, y_lo:y_hi],
#     radius=pixel_radius,
#     save=True,
#     figsize=(12, 12), # for comparison
# )
# _ = comparer.detect_b_to_a_thicken(radius=pixel_radius, DPI=600)
# _ = comparer.detect_a_to_b(radius=pixel_radius, DPI=600)
# _ = comparer.GPR_image()
# _ = comparer.GPR_lines()
# _ = comparer.crest_lines()
# _ = comparer.centerline_over_background(background=culled_background_flip[x_lo:x_hi, y_lo:y_hi].astype(np.uint8))